<a href="https://colab.research.google.com/github/AliceInHunterland/colab/blob/main/SpeechSplit_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/auspicious3000/SpeechSplit.git

Cloning into 'SpeechSplit'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 12), reused 34 (delta 9), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [4]:
cd SpeechSplit/

/content/SpeechSplit


In [5]:
cd assets/

/content/SpeechSplit/assets


In [6]:
!gdown --id {"1JF1WNS57wWcbmn1EztJxh09xU739j4_g"} -O model.zip

Downloading...
From: https://drive.google.com/uc?id=1JF1WNS57wWcbmn1EztJxh09xU739j4_g
To: /content/SpeechSplit/assets/model.zip
253MB [00:03, 75.0MB/s]


In [7]:
!gdown --id {"1SZPPnWAgpGrh0gQ7bXQJXXjOntbh4hmz"} -O autovc.ckpt

Downloading...
From: https://drive.google.com/uc?id=1SZPPnWAgpGrh0gQ7bXQJXXjOntbh4hmz
To: /content/SpeechSplit/assets/autovc.ckpt
341MB [00:05, 64.0MB/s]


In [8]:
!unzip model.zip

Archive:  model.zip
  inflating: 640000-P.ckpt           
  inflating: 660000-G.ckpt           


In [9]:
pip install wavenet_vocoder==0.1.1

  Created wheel for wavenet-vocoder: filename=wavenet_vocoder-0.1.1-cp36-none-any.whl size=12668 sha256=715be65bda79b937124fa83d11e7a290393a0af527c0557d58fc639f43e8bf06
  Stored in directory: /root/.cache/pip/wheels/72/fc/21/02d3785b65dd072b110b44b9df98b8cbf72a89ddea424ff0d9
Successfully built wavenet-vocoder


In [10]:
!git clone https://github.com/auspicious3000/autovc.git

Cloning into 'autovc'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 95 (delta 10), reused 60 (delta 3), pack-reused 23
Unpacking objects: 100% (95/95), done.


In [11]:
cd /content/SpeechSplit

/content/SpeechSplit


In [12]:
# demo conversion
import torch
import pickle
import numpy as np
from hparams import hparams
from utils import pad_seq_to_2
from utils import quantize_f0_numpy
from model import Generator_3 as Generator
from model import Generator_6 as F0_Converter


device = 'cuda:0'
G = Generator(hparams).eval().to(device)
g_checkpoint = torch.load('assets/660000-G.ckpt', map_location=lambda storage, loc: storage)
G.load_state_dict(g_checkpoint['model'])

P = F0_Converter(hparams).eval().to(device)
p_checkpoint = torch.load('assets/640000-P.ckpt', map_location=lambda storage, loc: storage)
P.load_state_dict(p_checkpoint['model'])


metadata = pickle.load(open('assets/demo.pkl', "rb"))


sbmt_i = metadata[0]
emb_org = torch.from_numpy(sbmt_i[1]).to(device)
x_org, f0_org, len_org, uid_org = sbmt_i[2]        
uttr_org_pad, len_org_pad = pad_seq_to_2(x_org[np.newaxis,:,:], 192)
uttr_org_pad = torch.from_numpy(uttr_org_pad).to(device)
f0_org_pad = np.pad(f0_org, (0, 192-len_org), 'constant', constant_values=(0, 0))
f0_org_quantized = quantize_f0_numpy(f0_org_pad)[0]
f0_org_onehot = f0_org_quantized[np.newaxis, :, :]
f0_org_onehot = torch.from_numpy(f0_org_onehot).to(device)
uttr_f0_org = torch.cat((uttr_org_pad, f0_org_onehot), dim=-1)

sbmt_j = metadata[1]
emb_trg = torch.from_numpy(sbmt_j[1]).to(device)
x_trg, f0_trg, len_trg, uid_trg = sbmt_j[2]        
uttr_trg_pad, len_trg_pad = pad_seq_to_2(x_trg[np.newaxis,:,:], 192)
uttr_trg_pad = torch.from_numpy(uttr_trg_pad).to(device)
f0_trg_pad = np.pad(f0_trg, (0, 192-len_trg), 'constant', constant_values=(0, 0))
f0_trg_quantized = quantize_f0_numpy(f0_trg_pad)[0]
f0_trg_onehot = f0_trg_quantized[np.newaxis, :, :]
f0_trg_onehot = torch.from_numpy(f0_trg_onehot).to(device)

with torch.no_grad():
    f0_pred = P(uttr_org_pad, f0_trg_onehot)[0]
    f0_pred_quantized = f0_pred.argmax(dim=-1).squeeze(0)
    f0_con_onehot = torch.zeros((1, 192, 257), device=device)
    f0_con_onehot[0, torch.arange(192), f0_pred_quantized] = 1
uttr_f0_trg = torch.cat((uttr_org_pad, f0_con_onehot), dim=-1)    


conditions = ['R', 'F', 'U', 'RF', 'RU', 'FU', 'RFU']
spect_vc = []
with torch.no_grad():
    for condition in conditions:
        if condition == 'R':
            x_identic_val = G(uttr_f0_org, uttr_trg_pad, emb_org)
        if condition == 'F':
            x_identic_val = G(uttr_f0_trg, uttr_org_pad, emb_org)
        if condition == 'U':
            x_identic_val = G(uttr_f0_org, uttr_org_pad, emb_trg)
        if condition == 'RF':
            x_identic_val = G(uttr_f0_trg, uttr_trg_pad, emb_org)
        if condition == 'RU':
            x_identic_val = G(uttr_f0_org, uttr_trg_pad, emb_trg)
        if condition == 'FU':
            x_identic_val = G(uttr_f0_trg, uttr_org_pad, emb_trg)
        if condition == 'RFU':
            x_identic_val = G(uttr_f0_trg, uttr_trg_pad, emb_trg)
            
        if 'R' in condition:
            uttr_trg = x_identic_val[0, :len_trg, :].cpu().numpy()
        else:
            uttr_trg = x_identic_val[0, :len_org, :].cpu().numpy()
                
        spect_vc.append( ('{}_{}_{}_{}'.format(sbmt_i[0], sbmt_j[0], uid_org, condition), uttr_trg ) )       

In [13]:
!touch /content/SpeechSplit/assets/autovc/__init__.py
!touch /content/SpeechSplit/assets/__init__.py
!touch /content/SpeechSplit/__init__.py
!cd /content/SpeechSplit/assets/autovc


In [14]:
cd /content/SpeechSplit/assets/autovc

/content/SpeechSplit/assets/autovc


In [ ]:
ls

In [16]:
!mv hparams.py hparam.py

In [17]:
cd /content/SpeechSplit

/content/SpeechSplit


In [18]:
ls

assets/         hparams.py   main.py           model.py      solver.py
data_loader.py  __init__.py  make_metadata.py  __pycache__/  tfcompat/
demo.ipynb      LICENSE      make_spect_f0.py  README.md     utils.py


 

---

---




 in synthesis.py change imports to 



```
from assets.autovc.hparam import hparams
```




---



In [ ]:
!pip install numba==0.48
!pip install librosa==0.6.3

In [20]:
import torch
import librosa
import pickle
import os
import soundfile as sf
print(librosa.__version__)

0.6.3


In [ ]:

# spectrogram to waveform
import torch
import librosa
import pickle
import os
import soundfile as sf
from assets.autovc.synthesis import build_model, wavegen
#from  os
#from speechsplit.assets.autovc. import synthesis
#from assets.autovc import *
if not os.path.exists('results'):
    os.makedirs('results')

model = build_model().to(device)
#checkpoint = torch.load("/content/SpeechSplit/assets/checkpoint_step001000000_ema.pth")
#model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)   
    librosa.output.write_wav('results/'+name+'.wav', waveform, sr=16000)
    #sf.write('results/'+name+'.wav', waveform,16000 ,'PCM_24')

  0%|          | 3/26880 [00:00<15:27, 28.97it/s]

p226_p231_003002_R


  0%|          | 8/34560 [00:00<08:04, 71.31it/s]

p226_p231_003002_F


  0%|          | 7/34560 [00:00<08:52, 64.83it/s]

p226_p231_003002_U


 68%|██████▊   | 23620/34560 [05:39<03:08, 57.96it/s]